# Module 5 - Lab 2 - Deployment - Docker 
## Pre-requisites
> handled by codespace custom initialization
- ollama installed and running 
- python packages 
    - langchain
    - langchain-openai
    - langgraph
- docker installed

In [ ]:
# Install packages for deployment and SDK
!pip install "langgraph-cli[inmem]" langgraph-sdk

## Step 1 - Clean-up and regenerate chat_app

In [ ]:
!rm -rf chat_app

In [ ]:
!!langgraph new --template react-agent-python chat_app

## Step 2 - setup .env (in chat_app dir)
- OPENAI_API_KEY not needed if using ollama option
```
LANGCHAIN_TRACING_V2=true
LANGCHAIN_PROJECT=005_deploy_lab
LANGCHAIN_API_KEY=xx
TAVILY_API_KEY=xx
OPENAI_API_KEY=xx
```

## Step 3 - Use LangGraph CLI to generate docker with compose

In [ ]:
%cd chat_app

In [ ]:
!langgraph dockerfile --help

In [ ]:
# LangGraph CLI to generate 
# - docker file call `Dockerfile` 
# - docker-compose.yml file
!langgraph dockerfile --add-docker-compose Dockerfile

## Step 4 - Startup Docker

In [ ]:
!docker compose up -d

In [ ]:
!docker ps

## Step 5 - Connect via SDK

In [12]:
# Connect via SDK
# - change to correct port
url_for_cli_deployment = "http://localhost:8123"

In [15]:
from langchain_core.messages import HumanMessage
from langgraph_sdk import get_client

client = get_client(url=url_for_cli_deployment)
config = {"configurable": {"user_id": "user1", "model": "openai/gpt-4o"}}
graph_name = "agent" 
thread = await client.threads.create()
user_input = "Who won the last superbowl?"
async for chunk in client.runs.stream(thread["thread_id"], 
                                      graph_name, 
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      config=config,
                                      stream_mode="messages-tuple"):

    if chunk.event == "messages":
        print("".join(data_item['content'] for data_item in chunk.data if 'content' in data_item), end="", flush=True)

[{"title": "Super Bowl 2025: Philadelphia Eagles Win, Defeat Kansas City Chiefs", "url": "https://www.facebook.com/enews/videos/super-bowl-2025-philadelphia-eagles-win-defeat-kansas-city-chiefs/446408351870404/", "content": "Super Bowl 2025: Philadelphia Eagles Win, Defeat Kansas City Chiefs | Patrick Mahomes, Travis Kelce and the rest of the Kansas City Chiefs tried their best to make history, but Jalen Hurts and the Philadelphia Eagles came... Super Bowl 2025: Philadelphia Eagles Win, Defeat Kansas City Chiefs E! News E! News 6.3K E! News 30K E! News 367K E! News 726K E! News 204K E! News 226K E! News 655K E! News 216K E! News 10h ago E! News 10h ago E! News 14h ago E! News 15h ago E! News 15h ago E! News 16h ago E! News 16h ago E! News E! News Entertainment website E! News Super Bowl 2025: Philadelphia Eagles Win, Defeat Kansas City Chiefs Forgot password?", "score": 0.8306082}, {"title": "Philadelphia Eagles become two-time NFL champions - YouTube", "url": "https://www.youtube.com/

## Shutdown

In [ ]:
!docker compose down